- General Setup:
    - 175 grid points
    - Cost function: $c(x, \theta_1) = 0.001 \theta_{11} x$
    - **Missing** true parameters: $RC=11.7257$, $\theta^0_{11}=2.4569$, $\theta^0_3=(0.0937, 0.4475, 0.4459, 0.0127, 0.0002)$, $\beta\in \{0.975, 0.985, 0.995, 0.999, 0.9995, 0.9999\}$
    - simulate $T=120$ (time periods) and $M=50$ (buses)
    - Monte Carlo with 250 data sets per different $\beta$
    - **Missing** Run NFXP and MPEC with the same varying starting values: $(RC^0=4, \theta^0_1=1), (RC^0=5, \theta^0_1=2), (RC^0=6, \theta^0_1=3), (RC^0=7, \theta^0_1=4), (RC^0=8, \theta^0_1=5)$ and in case of MPEC with the starting values $EV_1,...,EV_{75}=(0,...,0)$.
    - **Missing** Give out a table with the mean estimated parameters, their standard deviation and the MSE per $\beta$ and per method 
    - **Missing** Second table with the number of convergences, mean CPU time in seconds, mean number of major iterations, mean number of function evaluations, mean number of Bellman iterations, mean number of N-K iterations per $\beta$ and per method
    
    
- Specifications for NFXP:
    - Stopping tolerance of fixed point algorithm $10^{-13}$
    - Relative tolerance from contraction mapping to NK of $0.02$ and absolute of $0.01$
    - BHHH tolerance of $0.5$ $\rightarrow$ our tolerance is set up differently
    - Maximum number of steps in both contraction mapping and NK is $20$
    - Switching seems possible
    
    
- Specifications for MPEC:
    - constraint tolerance $10^{-6}$, relative tolerance on $x$ is $10^{-15}$ and the relative tolerance on the function values is $10^{-6}$.
    
    
- Differences:
    - We only run NFXP with partial likelihood approach. Not like in their paper where they first caluclate the partial likelihood and then the full again after.
    - They run the transition prob estimation with staring values based on frequency. We do not do that because the transition prob estimation is exactly the same for both NFXP and MPEC.
    - MPEC for them uses sparsity patterns of Hessian and Jacobian as well as analytical Hessian.

In [1]:
import numpy as np
import pandas as pd
import scipy.io
import time

from ruspy.estimation.estimation import estimate
from promotion.replication.auxiliary_iskhakov_et_al_2016.auxiliary import (process_data, process_result)

---
# Replication of Iskhakov et al. (2016)
---

In [10]:
# Initialize the simulation
discount_factor = [0.975, 0.985, 0.995, 0.999, 0.9995, 0.9999]
approach = ["NFXP", "MPEC", "MPEC (numerical)"]
starting_cost_params = np.vstack((np.arange(4,9), np.arange(1,6)))
starting_expected_value_fun = np.zeros(175)
number_runs = 250
number_buses = 50
number_periods = 120
number_states = 175
number_cost_params = 2

# Initialize the set up for the nested fixed point algorithm
stopping_crit_fixed_point = 1e-13
switch_tolerance_fixed_point = 1e-2

# Initialize the set up for MPEC
lower_bound = np.concatenate((np.full(number_states, -np.inf), np.full(number_cost_params, 0.0)))
upper_bound = np.concatenate((np.full(number_states, 50.0), np.full(number_cost_params, np.inf)))
rel_func_value_stopping_criteria = 1e-15
rel_x_value_stopping_criteria = 1e-12 

init_dict_nfxp = {
    "model_specifications": {
        "number_states": number_states,
        "maint_cost_func": "linear",
        "cost_scale": 1e-3,
    },
    "optimizer": {
        "approach": "NFXP",
        "algorithm": "estimagic_bhhh",
        "gradient": "Yes",
    },
    "alg_details": {
        "threshold": stopping_crit_fixed_point,
        "switch_tol": switch_tolerance_fixed_point,
    },
}

init_dict_mpec = {
    "model_specifications": {
        "number_states": number_states,
        "maint_cost_func": "linear",
        "cost_scale": 1e-3,
    },
    "optimizer": {
        "approach": "MPEC",
        "algorithm": "LD_SLSQP",
        "gradient": "Yes",
        "set_ftol_rel": rel_func_value_stopping_criteria,
        "set_xtol_rel": rel_x_value_stopping_criteria,
        "set_lower_bounds": lower_bound,
        "set_upper_bounds": upper_bound,
    },
}

In [11]:
# Initialize DataFrame to store the results of each run
index = pd.MultiIndex.from_product([discount_factor, 
                                    range(number_runs),
                                    range(starting_cost_params.shape[1]),
                                    approach],
                                   names=["Discount Factor", "Run", "Start", "Approach"])

columns=["RC", "theta_11", "theta_30", "theta_31", "theta_32", "theta_33",
         "CPU Time", "Converged", "# of Major Iter.", "# of Func. Eval.", 
         "# of Bellm. Iter.", "# of N-K Iter."]

results = pd.DataFrame(index=index, columns=columns)

In [12]:
discount_factor = [0.975, 0.985]

In [ ]:
# Main loop to calculate the results for each run
for factor in discount_factor:
    # load simulated data 
    mat = scipy.io.loadmat("auxiliary_iskhakov_et_al_2016/RustBusTableXSimDataMC250_beta" + str(int(100000*factor)))
    
    for run in range(number_runs):
        data = process_data(mat, run, number_buses, number_periods)
        
        for start in range(starting_cost_params.shape[1]):
            # Adapt the Initiation Dictionairy of NFXP for this run
            init_dict_nfxp["model_specifications"]["discount_factor"] = factor
            init_dict_nfxp["optimizer"]["params"] = pd.DataFrame(starting_cost_params[:, start], columns=["value"])
            
            # Run NFXP
            tic = time.perf_counter()
            transition_result_nfxp, cost_result_nfxp = estimate(init_dict_nfxp, data)
            toc = time.perf_counter()
            time_nfxp = toc - tic
            
            results.loc[factor, run, start, "NFXP"] = process_result(
                "NFXP", transition_result_nfxp, cost_result_nfxp, time_nfxp, number_states)
            
            # Adapt the Initiation Dictionairy of MPEC for this run
            init_dict_mpec["optimizer"]["gradient"] = "Yes"
            init_dict_mpec["model_specifications"]["discount_factor"] = factor
            init_dict_mpec["optimizer"]["params"] = np.concatenate((
                starting_expected_value_fun, starting_cost_params[:, start]))
            
            # Run MPEC
            tic = time.perf_counter()
            transition_result_mpec, cost_result_mpec = estimate(init_dict_mpec, data)
            toc = time.perf_counter()
            time_mpec = toc - tic         
            
            results.loc[
                factor, run, start, "MPEC"].loc[
                ~results.columns.isin(["# of Bellm. Iter.", "# of N-K Iter."])] = process_result(
                        "MPEC", transition_result_mpec, cost_result_mpec, time_mpec, number_states)
            
            # Adapt the Initiation Dictionairy of MPEC to use numerical first order derivatives
            init_dict_mpec["optimizer"]["gradient"] = "No"
            
            tic = time.perf_counter()
            transition_result_mpec_num, cost_result_mpec_num = estimate(init_dict_mpec, data)
            toc = time.perf_counter()
            time_mpec_num = toc - tic   
            
            results.loc[
                factor, run, start, "MPEC (numerical)"].loc[
                ~results.columns.isin(["# of Bellm. Iter.", "# of N-K Iter."])] = process_result(
                        "MPEC (numerical)", transition_result_mpec_num, cost_result_mpec_num, time_mpec_num, number_states)

In [49]:
# Let's have a look at the results DataFrame
results

RC theta_11   theta_30  \
Discount Factor Run Start Approach                                        
0.9750          0   0     NFXP              8.89422  1.55117  0.0988235   
                          MPEC              8.89424  1.55117  0.0988235   
                          MPEC (numerical)  8.89437  1.55123  0.0988235   
                    1     NFXP              8.89422  1.55117  0.0988235   
                          MPEC              8.89244  1.55085  0.0988235   
...                                             ...      ...        ...   
0.9999          249 3     MPEC                  NaN      NaN        NaN   
                          MPEC (numerical)      NaN      NaN        NaN   
                    4     NFXP                  NaN      NaN        NaN   
                          MPEC                  NaN      NaN        NaN   
                          MPEC (numerical)      NaN      NaN        NaN   

                                            theta_31  theta_32   theta_33  \
Discount Factor Run Start Approach                                          
0.9750          0   0     NFXP              0.442857  0.446555  0.0112605   
                          MPEC              0.442857  0.446555  0.0112605   
                          MPEC (numerical)  0.442857  0.446555  0.0112605   
                    1     NFXP              0.442857  0.446555  0.0112605   
                          MPEC              0.442857  0.446555  0.0112605   
...                                              ...       ...        ...   
0.9999          249 3     MPEC                   NaN       NaN        NaN   
                          MPEC (numerical)       NaN       NaN        NaN   
                    4     NFXP                   NaN       NaN        NaN   
                          MPEC                   NaN       NaN        NaN   
                          MPEC (numerical)       NaN       NaN        NaN   

                                            CPU Time Converged  \
Discount Factor Run Start Approach                               
0.9750          0   0     NFXP               1.78943         1   
                          MPEC                 2.369         1   
                          MPEC (numerical)   2.36695         1   
                    1     NFXP              0.898596         1   
                          MPEC               1.15174         1   
...                                              ...       ...   
0.9999          249 3     MPEC                   NaN       NaN   
                          MPEC (numerical)       NaN       NaN   
                    4     NFXP                   NaN       NaN   
                          MPEC                   NaN       NaN   
                          MPEC (numerical)       NaN       NaN   

                                           # of Major Iter. # of Func. Eval.  \
Discount Factor Run Start Approach                                             
0.9750          0   0     NFXP                            7                8   
                          MPEC                           16               16   
                          MPEC (numerical)               19               19   
                    1     NFXP                            7                8   
                          MPEC                           17               17   
...                                                     ...              ...   
0.9999          249 3     MPEC                          NaN              NaN   
                          MPEC (numerical)              NaN              NaN   
                    4     NFXP                          NaN              NaN   
                          MPEC                          NaN              NaN   
                          MPEC (numerical)              NaN              NaN   

                                           # of Bellm. Iter. # of N-K Iter.  
Discount Factor Run Start Approach                                           
0.9750         

In [56]:
results.to_pickle("results")

---
### Table I from Su & Judd (2012)
---

In [50]:
# Create Table I from Su & Judd (2012) with the simulated values from Iskahkov et al. (2016)
columns_table_1 = ["RC", "theta_11", "theta_30", "theta_31", "theta_32", "theta_33"]
table_1_temp = results[columns_table_1].astype(float).groupby(level=["Discount Factor", "Approach"])
statistic = ["Mean", "Standard Deviation"]
index = pd.MultiIndex.from_product([discount_factor, approach, statistic],
                                   names=["Discount Factor", "Approach", "Statistic"])

table_1 = pd.DataFrame(index=index, columns=columns_table_1)
table_1.loc(axis=0)[:,:,"Mean"] = table_1_temp.mean()
table_1.loc(axis=0)[:,:,"Standard Deviation"] = table_1_temp.std()

In [51]:
table_1

RC  theta_11  \
Discount Factor Approach         Statistic                               
0.975           NFXP             Mean                11.9079   2.50679   
                                 Standard Deviation   1.5173  0.468413   
                MPEC             Mean                 11.908   2.50681   
                                 Standard Deviation  1.51739  0.468435   
                MPEC (numerical) Mean                11.9084   2.50695   
                                 Standard Deviation  1.51757   0.46849   
0.985           NFXP             Mean                11.9865   2.53371   
                                 Standard Deviation  1.45744  0.452172   
                MPEC             Mean                11.9866   2.53373   
                                 Standard Deviation  1.45747  0.452179   
                MPEC (numerical) Mean                11.9877   2.53409   
                                 Standard Deviation  1.45793  0.452317   

                                                       theta_30    theta_31  \
Discount Factor Approach         Statistic                                    
0.975           NFXP             Mean                 0.0937022    0.447701   
                                 Standard Deviation  0.00377701  0.00642229   
                MPEC             Mean                 0.0937022    0.447701   
                                 Standard Deviation  0.00377701  0.00642229   
                MPEC (numerical) Mean                 0.0937022    0.447701   
                                 Standard Deviation  0.00377701  0.00642229   
0.985           NFXP             Mean                 0.0937022    0.447701   
                                 Standard Deviation  0.00377701  0.00642229   
                MPEC             Mean                 0.0937022    0.447701   
                                 Standard Deviation  0.00377701  0.00642229   
                MPEC (numerical) Mean                 0.0937022    0.447701   
                                 Standard Deviation  0.00377701  0.00642229   

                                                       theta_32    theta_33  
Discount Factor Approach         Statistic                                   
0.975           NFXP             Mean                  0.445587   0.0128134  
                                 Standard Deviation  0.00657434  0.00146026  
                MPEC             Mean                  0.445587   0.0128134  
                                 Standard Deviation  0.00657434  0.00146026  
                MPEC (numerical) Mean                  0.445587   0.0128134  
                                 Standard Deviation  0.00657434  0.00146026  
0.985           NFXP             Mean                  0.445587   0.0128134  
                                 Standard Deviation  0.00657434  0.00146026  
                MPEC             Mean                  0.445587   0.0128134  
                                 Standard Deviation  0.00657434  0.00146026  
                MPEC (numerical) Mean                  0.445587   0.0128134  
                                 Standard Deviation  0.00657434  0.00146026

---
### Corresponding (reduced) results from Iskhakov et al. (2016)
---

In [52]:
# Create a Table with the results 
index = pd.MultiIndex.from_product([discount_factor, statistic],
                                   names=["Discount Factor", "Statistic"])
NFXP_Iskhakov = pd.DataFrame(index=index, columns=["RC", "theta_11"])
for factor in discount_factor:
    NFXP_Iskhakov_temp = scipy.io.loadmat(
        "auxiliary_iskhakov_et_al_2016/new_solution_beta_" + str(int(100000*factor)))[
        "result_jr87_" + str(int(100000*factor))]
    NFXP_Iskhakov.loc[factor, "Mean"] = NFXP_Iskhakov_temp.mean(axis=0)
    NFXP_Iskhakov.loc[factor, "Standard Deviation"] = NFXP_Iskhakov_temp.std(axis=0)

In [53]:
NFXP_Iskhakov

RC  theta_11
Discount Factor Statistic                            
0.975           Mean                11.9135   2.50835
                Standard Deviation  1.51658  0.468155
0.985           Mean                11.9911   2.53496
                Standard Deviation   1.4572  0.452003

---
### Table I from Iskhakov et al. (2016)
---

In [54]:
# Create Table I from Iskhakov et al. (2016)
columns_table_2 = ["CPU Time", "Converged", "# of Major Iter.", "# of Func. Eval.", "# of Bellm. Iter.", "# of N-K Iter."]
table_2 = results[columns_table_2].astype(float).groupby(["Discount Factor", "Approach"]).mean()
#table_2["Converged"] = (table_2["Converged"]*number_runs*starting_cost_params.shape[1]).astype(int)

In [55]:
table_2

CPU Time  Converged  # of Major Iter.  \
Discount Factor Approach                                                  
0.9750          MPEC              1.559710        1.0           22.1032   
                MPEC (numerical)  3.471216        1.0           28.2520   
                NFXP              1.428674        1.0           11.7552   
0.9850          MPEC              1.813826        1.0           23.9080   
                MPEC (numerical)  3.876681        1.0           30.5440   
                NFXP              1.559418        1.0           11.3560   
0.9950          MPEC                   NaN        NaN               NaN   
                MPEC (numerical)       NaN        NaN               NaN   
                NFXP                   NaN        NaN               NaN   
0.9990          MPEC                   NaN        NaN               NaN   
                MPEC (numerical)       NaN        NaN               NaN   
                NFXP                   NaN        NaN               NaN   
0.9995          MPEC                   NaN        NaN               NaN   
                MPEC (numerical)       NaN        NaN               NaN   
                NFXP                   NaN        NaN               NaN   
0.9999          MPEC                   NaN        NaN               NaN   
                MPEC (numerical)       NaN        NaN               NaN   
                NFXP                   NaN        NaN               NaN   

                                  # of Func. Eval.  # of Bellm. Iter.  \
Discount Factor Approach                                                
0.9750          MPEC                       22.1032                NaN   
                MPEC (numerical)           28.2520                NaN   
                NFXP                       14.1216           301.7328   
0.9850          MPEC                       23.9080                NaN   
                MPEC (numerical)           30.5440                NaN   
                NFXP                       13.5808           291.1872   
0.9950          MPEC                           NaN                NaN   
                MPEC (numerical)               NaN                NaN   
                NFXP                           NaN                NaN   
0.9990          MPEC                           NaN                NaN   
                MPEC (numerical)               NaN                NaN   
                NFXP                           NaN                NaN   
0.9995          MPEC                           NaN                NaN   
                MPEC (numerical)               NaN                NaN   
                NFXP                           NaN                NaN   
0.9999          MPEC                           NaN                NaN   
                MPEC (numerical)               NaN                NaN   
                NFXP                           NaN                NaN   

                                  # of N-K Iter.  
Discount Factor Approach                          
0.9750          MPEC                         NaN  
                MPEC (numerical)             NaN  
                NFXP                    104.3992  
0.9850          MPEC                         NaN  
                MPEC (numerical)             NaN  
                NFXP                    105.6200  
0.9950          MPEC                         NaN  
                MPEC (numerical)             NaN  
                NFXP                         NaN  
0.9990          MPEC                         NaN  
                MPEC (numerical)             NaN  
                NFXP                         NaN  
0.9995          MPEC                         NaN  
                MPEC (numerical)             NaN  
                NFXP                         NaN  
0.9999          MPEC                         NaN  
                MPEC (numerical)             NaN  
                NFXP                         NaN